| ![EEW logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/eew.jpg?raw=true) | ![EDGI logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/edgi.png?raw=true) |
|---|---|

#### This notebook is licensed under GPL 3.0. Please visit our Github repo for more information: 
#### The notebook was collaboratively authored by the Environmental Data & Governance Initiative (EDGI) following our authorship protocol: https://docs.google.com/document/d/1CtDN5ZZ4Zv70fHiBTmWkDJ9mswEipX6eCYrwicP66Xw/
#### For more information about this project, visit https://www.environmentalenforcementwatch.org/

## How to Run this Notebook
* If you click on a gray **code** cell, a little “play button” arrow appears on the left. If you click the play button, it will run the code in that cell (“**running** a cell”). The button will animate. When the animation stops, the cell has finished running.
![Where to click to run the cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/pressplay.JPG?raw=true)
* You may get a warning that the notebook was not authored by Google. We know, we authored them! It’s okay. Click “Run Anyway” to continue. 
![Error Message](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/warning-message.JPG?raw=true)
* Run all of the cells in a Notebook to make a complete report. Please feel free to look at and **learn about each result as you create it**!

---

# Watershed statistics from ECHO

Here we load some helper code to get us going.

In [ ]:
!git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git -b watershed-geo
!pip install geopandas
import geopandas as geopandas
!pip install topojson
import topojson as tp

%run ECHO_modules/DataSet.py
%run ECHO_modules/utilities.py
%run ECHO_modules/make_data_sets.py
print("Done!")

Select the state you want to look more closely at.

In [ ]:
from ECHO_modules import geographies

state = widgets.Dropdown(
    options=states,
    value=states[0],
    description='State:',
    disabled=False,
)
state

Run this cell to load watershed data. It may take some time depending on the state you're looking at!

In [ ]:
# Load data from SBU database
# https://gis.stackexchange.com/questions/112057/sql-query-to-have-a-complete-geojson-feature-from-postgis
sql = """
    SELECT jsonb_build_object(
        'type', 'FeatureCollection', 'features', jsonb_agg(features.feature)
    )
    FROM (
        SELECT jsonb_build_object(
            'type', 'Feature','id', gid, 'geometry',
            ST_AsGeoJSON(geom)::jsonb,'properties',
            to_jsonb(inputs) - 'gid' - 'geom'
        ) AS feature
        FROM (
            SELECT *
            FROM "wbdhu8"
            WHERE "states" LIKE '%"""+state.value+"""%'
        ) inputs
    ) features;
"""
url= 'http://portal.gss.stonybrook.edu/echoepa/index2.php?query=' # Old server: 'http://apps.tlt.stonybrook.edu/echoepa/index2.php?query='
data_location=url+urllib.parse.quote_plus(sql) + '&pg'
#print(sql) # For debugging
print(data_location) # For debugging
gdf=None
try:
  gdf = geopandas.read_file(data_location)
  print("Data loaded. Now to map it!")
except:
  print('Something went wrong!')

Map the watersheds in this state. It may take some time depending on the state you're looking at! It may also "disconnect" if there are a lot of watersheds. Still, you can just skip to the next step ("Pick a specific watershed...")

In [ ]:
# Load map data
import json

x = tp.Topology(gdf, toposimplify=.01) # Simplify and topologize the watershed boundaries in order to reduce the file size
x = x.to_json() # save as topojson
y = json.loads(x) # load as json

# create the map
m = folium.Map()
w = folium.TopoJson(
    y,
    'objects.data',
    name = "Watersheds",
).add_to(m)
folium.GeoJsonTooltip(fields=["name"]).add_to(w)

# compute boundaries so that the map automatically zooms in
b1 = [list(gdf.total_bounds)[1],list(gdf.total_bounds)[0]]
b2 = [list(gdf.total_bounds)[3],list(gdf.total_bounds)[2]]
bounds = [b1,b2] #m.fit_bounds([[52.193636, -2.221575], [52.636878, -1.139759]])
m.fit_bounds(bounds)

# display the map!
display(m)

Pick a specific watershed.

In [ ]:
try :
    gdf.set_index("name", inplace=True) #Edit to not alter the gdf
except KeyError:
    # If the cell is run twice the index is already set and "name" is not available
    pass
    
hucs = gdf.index.unique()
selection = widgets.Dropdown(
    options= hucs.sort_values(),
    description='Watershed:',
    disabled=False,
)
selection

Get basic information about facilities in this watershed

In [ ]:
huc = gdf.at[selection.value, "huc8"]
huc = float(huc) #string or integer?

# Get everything we will need from ECHO_EXPORTER in a single DB query.
# We can then use the full dataframe to specialize views of it.
full_echo_data = None

sql = 'select * from "ECHO_EXPORTER" where "FAC_ACTIVE_FLAG" = \'Y\' and "FAC_DERIVED_HUC" = ' + str(huc) + ''
try:
    # Don't index.
    full_echo_data = get_data( sql )
except pd.errors.EmptyDataError:
    print("\nThere are no EPA facilities for this query.\n")
full_echo_data

Map these facilities.

In [ ]:
if ( full_echo_data is not None ):
    map_of_facilities = mapper(full_echo_data) # Some errors not caught here...
    this_watershed = gdf.loc[gdf.index == selection.value]
    w = folium.GeoJson(
      this_watershed,
      name = "Watershed",
    ).add_to(map_of_facilities)
    display( map_of_facilities ) 
else:
    print( "There are no facilities in the watershed." )   

Are the facilities complying with the Clean Water Act? Top violators over the past 13 quarters.

In [ ]:
%run ECHO_modules/utilities.py

In [ ]:
df_violators = get_top_violators( full_echo_data, 'NPDES_FLAG', selection.value, "Watershed", 
        'CWA_13QTRS_COMPL_HISTORY', 'CWA_FORMAL_ACTION_COUNT', 90 )
display( chart_top_violators( df_violators, selection.value, "Watershed", 'CWA' ))

Get more detailed, program-specific data for these facilities (e.g. longer-term historical [non]compliance with the Clean Water Act). First, select the program.

In [ ]:
data_set_list = ['CWA Violations', 'CWA Inspections', 'CWA Penalties', 
                 '2020 Discharge Monitoring',] 
data_sets = make_data_sets( data_set_list )

data_set_widget = show_data_set_widget( data_sets ) 

#Note: we might consider limiting to CWA/SDWA given the watershed-focus of this notebook

Get the data from the Stony Brook University database. 

In [ ]:
program = data_sets[ data_set_widget.value ]
program_results = program.store_results( region_type="Watershed", region_value=huc, state=state.value )
program_data = None
if ( program_results is not None ):
    program_data = program_results.dataframe.copy()

    display( program_data )
else:
    print( "There is no data for this data set in this watershed.")

Pollutant discharge data. We will pull up what facilities reported discharging into this watershed between October 2019 through September 2020. 

In [ ]:
dmr_data_set = data_sets['2020 Discharge Monitoring']
dmr_results = dmr_data_set.store_results( region_type="Watershed", region_value=huc, state=state.value )
dmr_data = None
if ( dmr_results is not None ):
    dmr_data = dmr_results.dataframe.copy()

    display( dmr_data )
else:
    print( "There is no discharge monitoring data for this data set in this watershed.")

Select a specific pollutant

In [ ]:
pollutants = widgets.Dropdown(
    options= sorted(list(dmr_data["PARAMETER_DESC"].unique())),
    description='Pollutant:',
    disabled=False,
)
pollutants

In [ ]:
this_dmr_data = dmr_data.loc[dmr_data["PARAMETER_DESC"] == pollutants.value]

# Cataloguing missing information
print((this_dmr_data["DMR_VALUE_NMBR"].isna().sum() / len(this_dmr_data) ) *100) # Percent of DMR values for this pollutant missing
print(100* this_dmr_data.drop_duplicates(subset=['LIMIT_VALUE_ID'])["LIMIT_VALUE_NMBR"].isna().sum()/len(this_dmr_data.drop_duplicates(subset=['LIMIT_VALUE_ID']))) # percent of LIMIT_VALUE_NMBR not reported. Does not account for stays.
#NMBR_OF_SUBMISSION - The attribute stores the number of months for submitting the DMRs for the limit set (e.g., monthly = 1, semi-annually = 6, quarterly = 3); this data element will be blank for Unscheduled Limit Sets. Must be greater than or equal to NMBR_OF_REPORT and be divisible by NMBR_OF_REPORT.
#NMBR_OF_REPORT - The number of months in the monitoring period covered by the DMR (e.g., monthly = 1, quarterly = 3, semi-annually = 6). 
#For example, if NMBR_OF_REPORT is 3, there should be 4 quarterly reports here. If it is 6, there should be 2.
# [SEH] Temporarily comment out the next line, until the NMBR_OF_REPORT field gets added to the MVIEW.
# display(this_dmr_data.groupby(["LIMIT_VALUE_ID"])[['NMBR_OF_REPORT']].agg({"count", "first"})) #first is just a cheap way to record the actual NMBR_OF_REPORT value

# Cataloguing important numbers
print((len(this_dmr_data) / len(dmr_data)) *100) # share of all reports accounted for by this pollutant
print(str(this_dmr_data.drop_duplicates(subset=['LIMIT_VALUE_ID'])["LIMIT_VALUE_NMBR"].sum())+" "+this_dmr_data["LIMIT_UNIT_DESC"].unique()[0]) # total permitted value LIMIT_VALUE_NMBR. Assumes units are same.
print(np.nanmedian(this_dmr_data["EXCEEDENCE_PCT"])) #median percent over permitted value for this pollutant, excluding NaNs. If the output is nan, all exceedance values are NaN

To export the above data as a spreadsheet you can view in Excel, run the code block below. The fiel will appear in the "Files" tab on Google Colab (click the folder on the left hand side of the screen.

In [ ]:
dmr_data.to_csv("dmr_data.csv")